# Fiddling around with making Bungie API calls to get json data 

## [Error Codes](https://bungie-net.github.io/multi/schema_Exceptions-PlatformErrorCodes.html#schema_Exceptions-PlatformErrorCodes)
## [Manifest](https://bungie-net.github.io/multi/operation_get_Destiny2-GetDestinyManifest.html#operation_get_Destiny2-GetDestinyManifest)


In [55]:
import numpy as np
import pandas as pd
import requests
import os
import re
from bs4 import BeautifulSoup

In [3]:
class BungieData(object):

    def __init__(self, api_key, session=None):
        '''
        session: an object from login.py defining a user session by 
        logging into Bungie.net (oAuth verified)
        '''
        self.session = session
        self.api_key = api_key
        
    def get_playerByTagName(self, gamertag):
        site_call = "https://bungie.net/Platform/Destiny2/SearchDestinyPlayer/2/" + gamertag
        request = requests.get(site_call, 
                                headers={"X-API-Key":self.api_key})
        return request.json()['Response']
    
    def get_DestinyUserId(self, gamertag):
        info = self.get_playerByTagName(gamertag)
        return int(info[0]['membershipId'])

    def get_BungieUserId(self, membership_id):
        '''
        Use old Destiny endpoint for PSN user to get the BUNGIE membershipId for a user
        '''
        site_call = "https://bungie.net/Platform/User/GetMembershipsById/" + str(membership_id) + "/2/" 
        request = requests.get(site_call, 
                                headers={"X-API-Key":self.api_key})
        return int(request.json()['Response']['bungieNetUser']['membershipId'])
    
    def get_DestinyUserProfile(self, membership_id, components=[100]):
        '''
        Use new Destiny 2 endpoint for PSN player using the Destiny membershipId 
        '''
        components = "?components=" + ','.join([str(c) for c in components])
        site_call = "https://bungie.net/Platform/Destiny2/2/Profile/" + str(membership_id) + "/" + components
        request = requests.get(site_call, 
                                headers={"X-API-Key":self.api_key})
        return request.json()['Response']
    
    def get_postGameStats(self, game_id):
        site_call = "https://bungie.net/Platform/Destiny2/Stats/PostGameCarnageReport/" + str(game_id)
        request = requests.get(site_call, 
                                headers={"X-API-Key":self.api_key})
        return request.json()['Response']
    
    def get_Manifest(self):
        site_call = "https://bungie.net/Platform/Destiny2/Manifest/"
        request = requests.get(site_call, 
                                headers={"X-API-Key":self.api_key})
        return request.json()['Response']
    
    def get_PlayerStats(self, membership_id):
        site_call = "https://bungie.net/Platform/Destiny2/2/Account/" + str(membership_id) + "/Stats/"
        request = requests.get(site_call, 
                                headers={"X-API-Key":self.api_key})
        return request.json()['Response']
    
    def get_StatDefinitions(self):
        site_call = "https://bungie.net/Platform/Destiny2/Stats/Definition/"
        request = requests.get(site_call, 
                                headers={"X-API-Key":self.api_key})
        return request.json()['Response']
            

    
class DestinyTrackerData(object):
    
    def __init__(self):
        '''
        Grabs aggregated data on Destiny Tracker which is not readily available from the Bungie API
        '''
        self.site = 'https://destinytracker.com/'
        
    def get_PvPMeta(self):
        page = requests.get(self.site) 
        soup = BeautifulSoup(page.content, 'html.parser')
        print(soup.select(".dtr-gear-meta .dtr-gear-meta-list .dtr-gear-item .details .name"))

In [4]:
bungie = BungieData(api_key=os.environ["BUNGIE_API_KEY"]) # Never put your keys in code... export 'em!
destinytracker = DestinyTrackerData()

In [5]:
page = requests.get('https://destinytracker.com/') 
soup = BeautifulSoup(page.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <script type="text/javascript">
   window.NREUM||(NREUM={});NREUM.info = {"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"5dfabc9f6c","applicationID":"100948194","transactionName":"ZlBSMhcEWRdWUxFQC18afTAmSn8LWlUmVgpFR18KCQBFS35eAVwc","queueTime":0,"applicationTime":6,"agent":"","atts":""}
  </script>
  <script type="text/javascript">
   window.NREUM||(NREUM={}),__nr_require=function(e,t,n){function r(n){if(!t[n]){var o=t[n]={exports:{}};e[n][0].call(o.exports,function(t){var o=e[n][1][t];return r(o||t)},o,o.exports)}return t[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<n.length;o++)r(n[o]);return r}({1:[function(e,t,n){function r(){}function o(e,t,n){return function(){return i(e,[f.now()].concat(u(arguments)),t?null:this,n),t?void 0:this}}var i=e("handle"),a=e(2),u=e(3),c=e("ee").get("tracer"),f=e("loader"),s=NREUM;"undefined"==typeof window.newrelic&&(newrelic=s);var p=["setPageViewName","set

Unfortunately, we can't get the meta list from Destiny Tracker since they're calling some asynchr method to input the info

In [6]:
# Get PvP Meta weapons list
destinytracker.get_PvPMeta()

[<span class="name">{{ weapon.name }}</span>]


In [7]:
# Get my own Bungie MembershipID
Me = bungie.get_playerByTagName("BalancedSeeker6")
Me

[{'iconPath': '/img/theme/destiny/icons/icon_psn.png',
  'membershipType': 2,
  'membershipId': '4611686018453385415',
  'displayName': 'BalancedSeeker6'}]

In [73]:
# Get Destiny MembershipID
my_destiny_id = bungie.get_DestinyUserId("BalancedSeeker6")
my_destiny_id

4611686018453385415

In [72]:
# Get my BUNGIE membershipID from using my DESTINY membershipID found in the above call
# This is the number found on the profile page of a user on Bungie.net
my_bungie_id = bungie.get_BungieUserId(4611686018453385415)
my_bungie_id

11927477

### To use GetProfile, you'll need to lookup the user's Destiny membershipID by searching by gamertag. Found that in the get_DestinyUserId call : 4611686018453385415

In [10]:
My_Profile = bungie.get_DestinyUserProfile(4611686018453385415, components=[100,200])
My_Profile 

{'profile': {'data': {'userInfo': {'membershipType': 2,
    'membershipId': '4611686018453385415',
    'displayName': 'BalancedSeeker6'},
   'dateLastPlayed': '2018-06-14T08:03:49Z',
   'versionsOwned': 3,
   'characterIds': ['2305843009260518141',
    '2305843009289675361',
    '2305843009303434294']},
  'privacy': 1},
 'characters': {'data': {'2305843009260518141': {'membershipId': '4611686018453385415',
    'membershipType': 2,
    'characterId': '2305843009260518141',
    'dateLastPlayed': '2018-06-14T08:03:49Z',
    'minutesPlayedThisSession': '247',
    'minutesPlayedTotal': '12353',
    'light': 342,
    'stats': {'1935470627': 342,
     '2996146975': 2,
     '392767087': 9,
     '1943323491': 3},
    'raceHash': 2803282938,
    'genderHash': 2204441813,
    'classHash': 3655393761,
    'raceType': 1,
    'classType': 0,
    'genderType': 1,
    'emblemPath': '/common/destiny2_content/icons/60530e4d21f5a32bebdca82cb178c4bf.jpg',
    'emblemBackgroundPath': '/common/destiny2_cont

In [225]:
# Get a random single game's stats
game_stats = bungie.get_postGameStats(100)
game_stats

{u'activityDetails': {u'directorActivityHash': 1658347443,
  u'instanceId': u'100',
  u'isPrivate': False,
  u'mode': 2,
  u'modes': [7, 2],
  u'referenceId': 1658347443},
 u'entries': [{u'characterId': u'2305843009260533952',
   u'extended': {u'values': {u'precisionKills': {u'basic': {u'displayValue': u'55',
       u'value': 55.0}}},
    u'weapons': [{u'referenceId': 53159280,
      u'values': {u'uniqueWeaponKills': {u'basic': {u'displayValue': u'4',
         u'value': 4.0}},
       u'uniqueWeaponKillsPrecisionKills': {u'basic': {u'displayValue': u'75%',
         u'value': 0.75}},
       u'uniqueWeaponPrecisionKills': {u'basic': {u'displayValue': u'3',
         u'value': 3.0}}}},
     {u'referenceId': 4024037919,
      u'values': {u'uniqueWeaponKills': {u'basic': {u'displayValue': u'75',
         u'value': 75.0}},
       u'uniqueWeaponKillsPrecisionKills': {u'basic': {u'displayValue': u'65%',
         u'value': 0.6533333333333333}},
       u'uniqueWeaponPrecisionKills': {u'basic': {u'

In [226]:
manifest = bungie.get_Manifest()
manifest

{u'mobileAssetContentPath': u'/common/destiny2_content/sqlite/asset/asset_sql_content_30228cabcdcf681095c36fab1da69e32.content',
 u'mobileClanBannerDatabasePath': u'/common/destiny2_content/clanbanner/clanbanner_sql_content_12a291b583d3f5ce5c30a648fa58b482.content',
 u'mobileGearAssetDataBases': [{u'path': u'/common/destiny2_content/sqlite/asset/asset_sql_content_30228cabcdcf681095c36fab1da69e32.content',
   u'version': 0},
  {u'path': u'/common/destiny2_content/sqlite/asset/asset_sql_content_669cef97d6bb31f46d5703bcb9acb7bd.content',
   u'version': 1},
  {u'path': u'/common/destiny2_content/sqlite/asset/asset_sql_content_669cef97d6bb31f46d5703bcb9acb7bd.content',
   u'version': 2}],
 u'mobileGearCDN': {u'Gear': u'/common/destiny2_content/geometry/gear',
  u'Geometry': u'/common/destiny2_content/geometry/platform/mobile/geometry',
  u'PlateRegion': u'/common/destiny2_content/geometry/platform/mobile/plated_textures',
  u'Shader': u'/common/destiny2_content/geometry/platform/mobile/shad

## Get a random statistic from Player Stats

In [22]:
# All time stats for PvE for all characters
my_stats['mergedAllCharacters']['results']['allPvE']['allTime'].keys()

dict_keys(['activitiesCleared', 'activitiesEntered', 'assists', 'totalDeathDistance', 'averageDeathDistance', 'totalKillDistance', 'kills', 'averageKillDistance', 'secondsPlayed', 'deaths', 'averageLifespan', 'bestSingleGameKills', 'bestSingleGameScore', 'opponentsDefeated', 'efficiency', 'killsDeathsRatio', 'killsDeathsAssists', 'objectivesCompleted', 'precisionKills', 'resurrectionsPerformed', 'resurrectionsReceived', 'score', 'heroicPublicEventsCompleted', 'adventuresCompleted', 'suicides', 'weaponKillsAutoRifle', 'weaponKillsFusionRifle', 'weaponKillsHandCannon', 'weaponKillsTraceRifle', 'weaponKillsPulseRifle', 'weaponKillsRocketLauncher', 'weaponKillsScoutRifle', 'weaponKillsShotgun', 'weaponKillsSniper', 'weaponKillsSubmachinegun', 'weaponKillsRelic', 'weaponKillsSideArm', 'weaponKillsSword', 'weaponKillsAbility', 'weaponKillsGrenade', 'weaponKillsGrenadeLauncher', 'weaponKillsSuper', 'weaponKillsMelee', 'weaponBestType', 'allParticipantsCount', 'allParticipantsScore', 'allParti

In [23]:
# All time stats for PvP for all characters
my_stats['mergedAllCharacters']['results']['allPvP']['allTime'].keys()

dict_keys(['activitiesEntered', 'activitiesWon', 'assists', 'totalDeathDistance', 'averageDeathDistance', 'totalKillDistance', 'kills', 'averageKillDistance', 'secondsPlayed', 'deaths', 'averageLifespan', 'score', 'averageScorePerKill', 'averageScorePerLife', 'bestSingleGameKills', 'bestSingleGameScore', 'opponentsDefeated', 'efficiency', 'killsDeathsRatio', 'killsDeathsAssists', 'objectivesCompleted', 'precisionKills', 'resurrectionsPerformed', 'resurrectionsReceived', 'suicides', 'weaponKillsAutoRifle', 'weaponKillsFusionRifle', 'weaponKillsHandCannon', 'weaponKillsTraceRifle', 'weaponKillsPulseRifle', 'weaponKillsRocketLauncher', 'weaponKillsScoutRifle', 'weaponKillsShotgun', 'weaponKillsSniper', 'weaponKillsSubmachinegun', 'weaponKillsRelic', 'weaponKillsSideArm', 'weaponKillsSword', 'weaponKillsAbility', 'weaponKillsGrenade', 'weaponKillsGrenadeLauncher', 'weaponKillsSuper', 'weaponKillsMelee', 'weaponBestType', 'winLossRatio', 'allParticipantsCount', 'allParticipantsScore', 'allP

In [50]:
play_type = np.random.choice(['allPvP', 'allPvE'])
play_stats = my_stats['mergedAllCharacters']['results'][play_type]['allTime']
print("Play Type: {}\n{}".format(play_type, play_stats))

Play Type: allPvP
{'activitiesEntered': {'statId': 'activitiesEntered', 'basic': {'value': 381.0, 'displayValue': '381'}}, 'activitiesWon': {'statId': 'activitiesWon', 'basic': {'value': 168.0, 'displayValue': '168'}}, 'assists': {'statId': 'assists', 'basic': {'value': 1134.0, 'displayValue': '1134'}, 'pga': {'value': 2.9763779527559056, 'displayValue': '3.0'}}, 'totalDeathDistance': {'statId': 'totalDeathDistance', 'basic': {'value': 0.0, 'displayValue': '0.0'}}, 'averageDeathDistance': {'statId': 'averageDeathDistance', 'basic': {'value': 0.0, 'displayValue': '0.0'}}, 'totalKillDistance': {'statId': 'totalKillDistance', 'basic': {'value': 53280.0, 'displayValue': '53280.0'}}, 'kills': {'statId': 'kills', 'basic': {'value': 3592.0, 'displayValue': '3592'}, 'pga': {'value': 9.427821522309712, 'displayValue': '9.4'}}, 'averageKillDistance': {'statId': 'averageKillDistance', 'basic': {'value': 14.832962138084632, 'displayValue': '14.8'}}, 'secondsPlayed': {'statId': 'secondsPlayed', 'ba

In [53]:
rand_stat_key = np.random.choice(list(play_stats.keys()))
rand_stat_key

'longestSingleLife'

In [65]:
' '.join(re.findall('[A-Z]?[a-z]+|[A-Z]+(?=[A-Z]|$)', rand_stat_key))

'longest Single Life'

In [54]:
random_stat = play_stats[rand_stat_key]
random_stat

{'statId': 'longestSingleLife',
 'basic': {'value': 390.0, 'displayValue': '6m 30s'},
 'activityId': '985414982'}

In [49]:
random_stat['basic']['displayValue']

'2189'

In [66]:
alexaResponse = {
	"version": "1.0",
	"session": {
		"new": true,
		"sessionId": "amzn1.echo-api.session.0cb65040-bbbc-4814-9712-ff328a4ac5bc",
		"application": {
			"applicationId": "amzn1.ask.skill.a0f2d358-b375-492c-87c7-321b808ecba9"
		},
		"user": {
			"userId": "amzn1.ask.account.AHHWXHLTRBEBOOJVLVHDNXWTII4JPW6OUQPTZYPN4CKTO7QQWNVDP3T2GEANLDHYC5BXYBDMMMARFKKUTMD63OU6MCPD4AMKM6E4LK4L7SWO5L657W6UKP3ZJORKWMQT3AMV7EICAGRTFYC5NLHTQU7VBDIGOTNH43V3SETZ7L64SVVO7AKQF7A7R6TA7D2UARDPRUCLAD4IXRY"
		}
	},
	"context": {
		"AudioPlayer": {
			"playerActivity": "IDLE"
		},
		"Display": {},
		"System": {
			"application": {
				"applicationId": "amzn1.ask.skill.a0f2d358-b375-492c-87c7-321b808ecba9"
			},
			"user": {
				"userId": "amzn1.ask.account.AHHWXHLTRBEBOOJVLVHDNXWTII4JPW6OUQPTZYPN4CKTO7QQWNVDP3T2GEANLDHYC5BXYBDMMMARFKKUTMD63OU6MCPD4AMKM6E4LK4L7SWO5L657W6UKP3ZJORKWMQT3AMV7EICAGRTFYC5NLHTQU7VBDIGOTNH43V3SETZ7L64SVVO7AKQF7A7R6TA7D2UARDPRUCLAD4IXRY"
			},
			"device": {
				"deviceId": "amzn1.ask.device.AFRJ3AWPBRE655ZPUKCIUNXBEOHEUARYVLTGWMFSA2PV4ML3T5GLQFNWNRZOUZXRJJATHKDT5HZ26A57WZ3ARC6OLOQBGJACU3WQ6UHS2KOX2IOTHP4Z72JH7QJ6PT5FSEUBHR4M57S7Z6DOPEMVHDLE7R3AURSW4REJ6PNOILRVEQUGKSUIG",
				"supportedInterfaces": {
					"AudioPlayer": {},
					"Display": {
						"templateVersion": "1.0",
						"markupVersion": "1.0"
					}
				}
			},
			"apiEndpoint": "https://api.amazonalexa.com",
			"apiAccessToken": "eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsImtpZCI6IjEifQ.eyJhdWQiOiJodHRwczovL2FwaS5hbWF6b25hbGV4YS5jb20iLCJpc3MiOiJBbGV4YVNraWxsS2l0Iiwic3ViIjoiYW16bjEuYXNrLnNraWxsLmEwZjJkMzU4LWIzNzUtNDkyYy04N2M3LTMyMWI4MDhlY2JhOSIsImV4cCI6MTUyOTAyMDUzMiwiaWF0IjoxNTI5MDE2OTMyLCJuYmYiOjE1MjkwMTY5MzIsInByaXZhdGVDbGFpbXMiOnsiY29uc2VudFRva2VuIjpudWxsLCJkZXZpY2VJZCI6ImFtem4xLmFzay5kZXZpY2UuQUZSSjNBV1BCUkU2NTVaUFVLQ0lVTlhCRU9IRVVBUllWTFRHV01GU0EyUFY0TUwzVDVHTFFGTldOUlpPVVpYUkpKQVRIS0RUNUhaMjZBNTdXWjNBUkM2T0xPUUJHSkFDVTNXUTZVSFMyS09YMklPVEhQNFo3MkpIN1FKNlBUNUZTRVVCSFI0TTU3UzdaNkRPUEVNVkhETEU3UjNBVVJTVzRSRUo2UE5PSUxSVkVRVUdLU1VJRyIsInVzZXJJZCI6ImFtem4xLmFzay5hY2NvdW50LkFISFdYSExUUkJFQk9PSlZMVkhETlhXVElJNEpQVzZPVVFQVFpZUE40Q0tUTzdRUVdOVkRQM1QyR0VBTkxESFlDNUJYWUJETU1NQVJGS0tVVE1ENjNPVTZNQ1BENEFNS002RTRMSzRMN1NXTzVMNjU3VzZVS1AzWkpPUktXTVFUM0FNVjdFSUNBR1JURllDNU5MSFRRVTdWQkRJR09UTkg0M1YzU0VUWjdMNjRTVlZPN0FLUUY3QTdSNlRBN0QyVUFSRFBSVUNMQUQ0SVhSWSJ9fQ.ds8IDaa-bNews04NNDjqawFhaRUv57Qe8i6z48xOrjxQXkBJVuskUyJ70dgjybsW6TFkYn1i_tiCk9V_tiJtKW3lwHJX5MS_ynNr346dZm0vePSj9a_pPIMJBqM02VVLSw9WVvL5BQgK22JXMoUUFBjxFLn9cFItWuQ5JBnTypPMkGIluQw-Y1h9whtzN8OU2u4zlE7tlNzAK0ceh2qhDGIQQmIJkBT_d7QV5TnRR9YO6UUTyn3LIXSRzs4o85b0hei6vjaPgILlCvpxi1G4QPvzz59yZ-178VNpFRC0gB_Rb1UBzrIOPXVxCGe5SX2A_e9KNVRXYxuY3RJ7iEnUzQ"
		}
	},
	"request": {
		"type": "IntentRequest",
		"requestId": "amzn1.echo-api.request.3fc30350-64ee-42b8-9d8f-67cbb5bc556b",
		"timestamp": "2018-06-14T22:55:32Z",
		"locale": "en-US",
		"intent": {
			"name": "GetRandomPlayerStat",
			"confirmationStatus": "NONE",
			"slots": {
				"PlayType": {
					"name": "PlayType",
					"confirmationStatus": "NONE"
				}
			}
		}
	}
}

NameError: name 'true' is not defined

In [20]:
def char_stats_json2df(json):
    for char in My_Stats['characters']:
        if not char['deleted']:
            char_id = int(char['characterId'])

statsdf = pd.DataFrame(My_Stats)
statsdf

""


In [234]:
stat_definitions = bungie.get_StatDefinitions()
stat_definitions

{u'activitiesCleared': {u'category': 0,
  u'group': 1,
  u'modes': [7, 6, 4, 2, 3, 17, 16, 18],
  u'periodTypes': [2, 1],
  u'statDescription': u'Successful completions of this mission type.',
  u'statId': u'activitiesCleared',
  u'statName': u'Activities Cleared',
  u'unitLabel': u'',
  u'unitType': 1,
  u'weight': 1},
 u'activitiesEntered': {u'category': 8,
  u'group': 1,
  u'modes': [5, 10, 12, 7, 6, 4, 2, 3, 17, 16, 18, 19, 37, 38, 31, 39],
  u'periodTypes': [2, 1],
  u'statDescription': u'Battles fought - some lost, some won.',
  u'statId': u'activitiesEntered',
  u'statName': u'Activities Entered',
  u'unitLabel': u'',
  u'unitType': 1,
  u'weight': 1},
 u'activitiesWon': {u'category': 0,
  u'group': 1,
  u'modes': [5, 10, 12, 31, 19, 38, 37, 39],
  u'periodTypes': [2, 1],
  u'statId': u'activitiesWon',
  u'statName': u'Wins',
  u'unitLabel': u'',
  u'unitType': 1,
  u'weight': 1},
 u'activityAssists': {u'category': 2,
  u'group': 102,
  u'modes': [5, 10, 12, 7, 6, 4, 2, 3, 17, 1

### Get Post Game Carnage Reports
>"If you want it specifically for you, you can use the *GetActivityHistory* endpoint to get all of your PGCR IDs (grab the **referenceId** property on the **DestinyHistoricalStatsActivity** objects being returned), and then use that to make calls to the *GetPostGameCarnageReport* endpoint!" - vthornheart-bng https://github.com/Bungie-net/api/issues/452